In [1]:
# Imports
import pandas as pd
import numpy as np
import networkx
box_dir = '/Users/plutzer/Box'

In [3]:
# Loading files
proteinGroups = pd.read_csv(box_dir + '/CellBio-MajorLab/Users/Rotation Students/Isaac/MS search outputs/12-9-2020_razorUnique/txt/proteinGroups.txt',delimiter = '\t')
proteinGroups

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (222,223) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Fasta headers,Number of proteins,Peptides,Razor + unique peptides,Unique peptides,...,id,Peptide IDs,Peptide is razor,Mod. peptide IDs,Evidence IDs,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Oxidation (M) site positions,Taxonomy IDs
0,CON__ENSEMBL:ENSBTAP00000001528,CON__ENSEMBL:ENSBTAP00000001528,12,1,1,NaN,1,12,1,1,...,0,2345;4596;5737;14451;17699;21131;21424;26940;2...,False;False;False;False;True;False;False;False...,2474;4892;6114;15264;18736;22336;22643;28943;2...,19649;40547;40548;49776;49777;49778;49779;4978...,23312;49161;49162;49163;49164;59802;59803;5980...,23312;49163;59804;142190;180568;215139;217235;...,0,539,-1
1,CON__ENSEMBL:ENSBTAP00000024146;sp|P01023|A2MG...,CON__ENSEMBL:ENSBTAP00000024146,11;2;1,11;2;1,11;2;1,NaN,3,11,11,11,...,1,3419;6067;13247;16268;16952;21884;27734;29856;...,True;True;True;True;True;True;True;True;True;T...,3639;6465;14003;17227;17968;23128;29773;32029;...,28705;28706;52109;52110;52111;52112;52113;5211...,34281;34282;62384;62385;62386;62387;62388;6238...,34281;62389;129144;164100;172699;221240;281881...,NaN,NaN,-1;-1;-1
2,CON__ENSEMBL:ENSBTAP00000038253,CON__ENSEMBL:ENSBTAP00000038253,12,4,2,NaN,1,12,4,2,...,2,737;2892;6477;10482;18142;20642;25225;26991;29...,True;False;False;False;False;False;True;True;T...,779;3079;6888;6889;11092;19205;21796;21797;271...,6424;24558;24559;24560;24561;24562;24563;24564...,7579;29385;29386;29387;29388;29389;29390;29391...,7579;29407;66564;101337;185793;210703;259545;2...,1;2,255;465,-1
3,CON__ENSEMBL:ENSP00000377550;sp|P13646|K1C13_H...,CON__ENSEMBL:ENSP00000377550;sp|P13646|K1C13_H...,13;13;12;3;3;1;1,5;5;5;0;0;0;0,4;4;4;0;0;0;0,";sp|P13646|K1C13_HUMAN Keratin, type I cytoske...",7,13,5,4,...,3,1374;1924;9544;15912;17936;18314;19002;21645;2...,True;False;True;True;False;False;False;True;Fa...,1440;2027;10114;10115;16820;18990;19388;20097;...,11595;16140;16141;16142;16143;16144;16145;1614...,13717;19202;19203;19204;19205;19206;19207;1920...,13717;19216;93020;158793;183796;187491;193120;...,NaN,NaN,-1;-1;-1;-1;-1;-1;-1
4,CON__H-INV:HIT000292931,CON__H-INV:HIT000292931,13,1,1,NaN,1,13,1,1,...,4,3039;6181;8330;9702;10480;18890;21843;22714;24...,False;False;False;False;False;False;False;True...,3231;6584;8838;10279;11090;19982;23085;23989;2...,25589;25590;25591;25592;25593;25594;25595;2559...,30654;30655;30656;30657;30658;30659;30660;3066...,30665;63825;82858;94172;101227;192451;220864;2...,3;4;5;6;7,96;99;208;216;338,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,sp|Q9Y6X2|PIAS3_HUMAN,sp|Q9Y6X2|PIAS3_HUMAN,2,2,2,sp|Q9Y6X2|PIAS3_HUMAN E3 SUMO-protein ligase P...,1,2,2,2,...,4794,6806;22168,True;True,7240;23423,58110;58111;58112;58113;58114;185786;185787;18...,69285;69286;69287;223702;223703;223704,69285;223702,NaN,NaN,-1
4795,sp|Q9Y6X4|F169A_HUMAN,sp|Q9Y6X4|F169A_HUMAN,6,6,6,sp|Q9Y6X4|F169A_HUMAN Soluble lamin-associated...,1,6,6,6,...,4795,10771;16147;21568;26291;32244;33939,True;True;True;True;True;True,11398;17099;22796;28251;34582;36385,88389;135604;181233;223315;270819;270820;284629,104114;162818;218475;218476;269436;269437;3257...,104114;162818;218476;269437;325738;342194,6679,301,-1
4796,sp|Q9Y6X8|ZHX2_HUMAN,sp|Q9Y6X8|ZHX2_HUMAN,7,7,7,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,1,7,7,7,...,4796,5735;12284;12403;22754;28188;33942;34524,True;True;True;True;True;True;True,6112;12990;13115;24030;30249;36388;37012,49769;101613;101614;101615;102555;190973;19097...,59794;120357;120358;121490;229864;229865;22986...,59794;120358;121490;229866;286268;342208;349392,NaN,NaN,-1
4797,sp|Q9Y6Y0|NS1BP_HUMAN,sp|Q9Y6Y0|NS1BP_HUMAN,1,1,1,sp|Q9Y6Y0|NS1BP_HUMAN Influenza virus NS1A-bin...,1,1,1,1,...,4797,22926,True,24209,192262;192263;192264;192265;192266;192267;1922...,231374;231375;231376;231377;231378;231379;2313...,231379,NaN,NaN,-1


In [21]:
# Get a list of all protein IDs in the groups
groupedIDs = list(proteinGroups['Protein IDs'])
proteinIDs = list(set([Ids for groups in groupedIDs for Ids in groups.split(';')]))
len(proteinIDs)

5424

In [46]:
# Make an adjacency matrix for the protein groups based on shared proteins:
# 0 = no shared proteins between groups
# 1 = shared proteins between groups
adjMat = np.zeros((len(groupedIDs),len(groupedIDs)))
for group_index in range(len(groupedIDs)):
    for protein in groupedIDs[group_index].split(';'):
        also_in  = list(np.where([protein in group for group in proteinGroups['Protein IDs']])[0])
        for index in also_in:
            if group_index != index:
                adjMat[group_index,index] = 1

In [52]:
np.sum(adjMat)

50.0

In [54]:
graph = networkx.from_numpy_matrix(adjMat)

In [59]:
subgraphs = [list(graph.subgraph(c).copy().nodes()) for c in networkx.connected_components(graph)]


In [60]:
#Final function for Part 1
def generate_linked_groups(groupedIDs):
    '''
        Takes a list of protein groups and generates linked protein groups based on shared proteins.
        Generates an adjacency matrix for protein groups where:
            0 = no shared proteins between groups
            1 = shared proteins between groups
        And then finds connected subgraphs for that network. Each subgraph is a set of linked groups.
        groupedIDs: list of strings - protein IDs column from protein groups file. Proteins within a group should be separated by ';'
        returns: list of lists of ints - each element is a list of indexes protein groups in a linked group
    '''
    adjMat = np.zeros((len(groupedIDs),len(groupedIDs))) #Initializing the adjacency matrix
    
    #Iterate over each protein group, and find other protein groups that contain the same proteins.
    for group_index in range(len(groupedIDs)):
        for protein in groupedIDs[group_index].split(';'):
            
            # also in is a list of indexes of protein groups that share a protein with this group.
            also_in  = list(np.where([protein in group for group in proteinGroups['Protein IDs']])[0])
            
            # Iterate through also_in and update the adjacency matrix
            for index in also_in:
                if group_index != index:
                    adjMat[group_index,index] = 1
    
    # Generate a list of indexes for component subgraphs
    graph = networkx.from_numpy_matrix(adjMat)
    return [list(graph.subgraph(c).copy().nodes()) for c in networkx.connected_components(graph)]
    

In [61]:
generate_linked_groups(list(proteinGroups['Protein IDs']))

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [337, 86],
 [421, 87],
 [88],
 [89, 527],
 [90],
 [91],
 [92],
 [760, 93, 327],
 [813, 94],
 [95],
 [96],
 [904, 97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104, 1268],
 [105, 1284, 4486],
 [106, 1358],
 [107, 1375],
 [108],
 [109, 1566],
 [110],
 [111],
 [112, 1808],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118, 2070],
 [2089, 119],
 [120, 1009],
 [2272, 121],
 [122],
 [123, 2303],
 [124],
 [1